In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import requests
import json
import movecolumn as mc
import os
import glob
import plotly.express as px

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

# import dataset
df = pd.read_csv('/content/Drive/MyDrive/Final_Project/segment_1_ETL/merged_violence_laws.csv')
df.head(5)

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).
Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


,Unnamed: 0,state,n_killed,n_injured,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
0,0,Alabama,488,761,2016,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,10
1,1,Alaska,88,103,2016,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,2,Arizona,277,294,2016,1,1,1,1,0,...,0,0,0,0,0,0,0,0,1,8
3,3,Arkansas,190,351,2016,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,11
4,4,California,1265,1896,2016,1,1,0,1,0,...,1,1,1,1,1,1,1,1,1,104


In [3]:
df = df.set_index('state')
df = df.reset_index(drop=False)
df = df.drop(['year','Unnamed: 0'], axis=1)
df.head(5)

,state,n_killed,n_injured,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
0,Alabama,488,761,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,10
1,Alaska,88,103,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,Arizona,277,294,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,8
3,Arkansas,190,351,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,11
4,California,1265,1896,1,1,0,1,0,0,1,...,1,1,1,1,1,1,1,1,1,104


In [4]:
# rename columns
df.rename(columns={'n_killed':'deaths', 'n_injured':'injuries'}, inplace=True)

In [5]:
df.deaths.describe()

count      50.000000
mean      299.200000
std       312.149101
min        13.000000
25%        68.500000
50%       206.000000
75%       426.750000
max      1313.000000
Name: deaths, dtype: float64

In [6]:
df['deaths_above_below_mean'] = np.where(df['deaths'] < 300, 'below_national_mean', 'above_national_mean')
df.head(5)

,state,deaths,injuries,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_above_below_mean
0,Alabama,488,761,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,10,above_national_mean
1,Alaska,88,103,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,below_national_mean
2,Arizona,277,294,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,8,below_national_mean
3,Arkansas,190,351,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,11,below_national_mean
4,California,1265,1896,1,1,0,1,0,0,1,...,1,1,1,1,1,1,1,1,104,above_national_mean


In [7]:
# move new column to between 'state' and 'deaths'
df = mc.MoveTo1(df,'deaths_above_below_mean')
df.head(5)

,deaths_above_below_mean,state,deaths,injuries,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
0,above_national_mean,Alabama,488,761,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,10
1,below_national_mean,Alaska,88,103,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,below_national_mean,Arizona,277,294,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,1,8
3,below_national_mean,Arkansas,190,351,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,11
4,above_national_mean,California,1265,1896,1,1,0,1,0,0,...,1,1,1,1,1,1,1,1,1,104


In [8]:
# determine state's total death rate as ranking 1 (lowest quartile death rate) to 4 (highest quartile death rate)
quantiles = pd.qcut(df['deaths'], 4, labels=False)
df = df.assign(quantiles=quantiles.values)

# move new column to between 'state' and 'deaths'
df = mc.MoveTo3(df,'quantiles')

# rename quantile values
df['quantiles'] = df['quantiles'].replace(['0', '1', '2', '3'], ['1', '2', '3', '4'])

df.rename(columns={'quantiles':'deaths_quantiles'}, inplace=True)

df.head(5)

,deaths_above_below_mean,state,deaths_quantiles,deaths,injuries,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
0,above_national_mean,Alabama,3,488,761,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,10
1,below_national_mean,Alaska,1,88,103,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,below_national_mean,Arizona,2,277,294,1,1,1,1,0,...,0,0,0,0,0,0,0,0,1,8
3,below_national_mean,Arkansas,1,190,351,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,11
4,above_national_mean,California,3,1265,1896,1,1,0,1,0,...,1,1,1,1,1,1,1,1,1,104


In [9]:
# determine state's total death rate as ranking 1 (lowest quartile death rate) to 4 (highest quartile death rate)
quantiles = pd.qcut(df['lawtotal'], 4, labels=False)
df = df.assign(quantiles=quantiles.values)

# move new column to between 'state' and 'deaths'
df = mc.MoveTo4(df,'quantiles')

# rename quantile values
df['quantiles'] = df['quantiles'].replace(['0', '1', '2', '3'], ['1', '2', '3', '4'])

df.rename(columns={'quantiles':'laws_quantiles'}, inplace=True)

df.head(5)

,deaths_above_below_mean,state,deaths_quantiles,laws_quantiles,deaths,injuries,felony,invcommitment,invoutpatient,danger,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
0,above_national_mean,Alabama,3,1,488,761,0,1,0,1,...,0,0,0,0,0,0,0,0,0,10
1,below_national_mean,Alaska,1,0,88,103,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,below_national_mean,Arizona,2,0,277,294,1,1,1,1,...,0,0,0,0,0,0,0,0,1,8
3,below_national_mean,Arkansas,1,1,190,351,1,1,0,1,...,0,0,0,0,0,0,0,0,0,11
4,above_national_mean,California,3,3,1265,1896,1,1,0,1,...,1,1,1,1,1,1,1,1,1,104


In [10]:

fig = px.bar(df, x='state', y='deaths')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

In [11]:
fig = px.bar(df, x='state', y='deaths_quantiles')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

In [12]:
fig = px.bar(df, x='state', y='lawtotal')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

In [14]:
fig = px.bar(df, x='state', y='laws_quantiles')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

In [16]:
model_df = df[["lawtotal", "deaths", "injuries","deaths_above_below_mean","state","laws_quantiles","deaths_quantiles"]]

In [19]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
model_df['total_laws'] = labelencoder.fit_transform(model_df['lawtotal'])
model_df["total_deaths"] = labelencoder.fit_transform(model_df['deaths'])
model_df["total_injurites"] = labelencoder.fit_transform(model_df['injuries'])
model_df["deaths_above_below_mean"].value_counts()

below_national_mean    32
above_national_mean    18
Name: deaths_above_below_mean, dtype: int64

In [20]:
X = model_df[["total_laws", "total_deaths"]]
y = model_df["deaths_above_below_mean"]

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
scaler = StandardScaler().fit(X_train)

In [22]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [24]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression()

In [25]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9


In [26]:
classifier.predict([[7,0]])

array(['above_national_mean'], dtype=object)

In [27]:
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.9


In [28]:
clf.predict([[1,0]])

array(['below_national_mean'], dtype=object)

In [29]:
model_df[["total_laws", "total_deaths", "deaths_above_below_mean"]].to_csv("db.csv", index=False)